In [ ]:
! pip install torch torchvision 
! pip install torch
! pip install matplotlib

In [ ]:
#Importamos las librerias necesarias
import os
import torch
from torch import optim, nn, utils, Tensor
#cuda
import torch.cuda

cosas por hacer 
 el análisis de un dataset y el entrenamiento de un modelo para la recomendación de cultivos a sembrar según la composición de un terreno y las condiciones climáticas del área.

 preprocesar el dataset segun sea lo necesario
    -podria se aqui mas como sacar bien los rangos de cada parametro segun la etiqueta
    -ademas de ver datos que esten fuera de la distribucion y matarlos para que no genere tanto ruido
    -ver como varia los datos entre cada labe pudiendo tener un minimo y un maximo
    -usar histogramas con pandas y regularizar datos

 para el entrenamiento 

1. Considerando el dataset anterior, implemente un bucle de entrenamiento de la tarea correspondiente.

2. Al terminar el entrenamiento del bucle, debe tener una función en la cual se le pase un vector de datos y retorne la inferencia correspondiente.

3. El bucle debe mostrar las curvas de las funciones de cost para entrenamiento y validación, así como las métricas relevantes.

4. A su vez, realizar comparaciones entre desempeño usando al menos tres arquitecturas de modelos distintas.
-podria ser usar softmax para sacar probabilidades
-

